This notebook is intended just for the SVG files in the paper:

"Automatic 3D reconstruction from 2D orthographic drawings"

Assumptions:

1. The structure of the XML files is as follows:
    
        xml info
        svg info
            title
            description
            definitions
            content group
                container group
                    edge container (contains transform and hidden line info)
                        edge projection
                    edge container
                        edge projection
                    .
                    .
                    .
                    edge container
                        edge projection
      
2. All projections are first angle:
    Left view is located to the right of the front view
    Bottom view is located on top of the front view
    etc.
3. The drawing contains exactly three views: Front, Bottom, and Left
4. Transforms specified in the edge container do not include skews or rotations. I am just extracting the translation part.
5. The transform translation information specifies the view, e.g:
    - Front view is
        
        (1485, 1050)
    - Bottom view is
        
        (1485, 700)
    - Left view is
        
        (1800, 1050)
6. All edge elements are given as paths.
7. All line elements are specified with absolute line units "L"
8. All curve elements are specified with absolute cubic Bezier units "C"
9. All curve elements are quarter circles (We'll see about this one)
    - I'm not sure how to calculate the bounding box of a bezier curve.
    - Quarter circles have a bounding box that is easy to calculate

In [ ]:
import re
import numpy as np
from xml.etree import ElementTree as ET
from typing import Tuple

LOOPMAX: int = 1000

In [ ]:
class View:
    def __init__(self, coords: Tuple[float, float]):
        self.coords = coords
        self.absolute = True
        self.b_box = ((None, None), (None, None))
        self.lines = []

    def append(self, item) -> None: # item should be a Move, Line, or Curve instance
        self.lines.append(item)

In [ ]:
class Move:
    def __init__(self, text: str):
        self.text: str = text

        # Split text by commas or spaces (both are valid delimiters)
        move_list: list = re.split(r"[, ]+", self.text)
        # Remove 'M' from first element to make it consist of just numbers, minus sign, and/or decimal points
        move_list[0] = re.sub("M", "", move_list[0])

        self.x: float = float(move_list[0])
        self.y: float = float(move_list[1])
        self.end_coords: tuple = (self.x, self.y)

In [ ]:
move = Move("M219.615,-410.764 ")

print(move.text)
print(f"x: {move.x:<8}\t y: {move.y}")

In [ ]:
class Line:
    def __init__(self, text: str, current_pt: Tuple[float, float], view_coords: Tuple[float, float], hidden=False):
        self.text: str = text

        # Split text by commas or spaces (both are valid delimiters)
        line_list: list = re.split(r"[, ]+", self.text)
        # Remove 'M' from first element to make it consist of just numbers, minus sign, and/or decimal points
        line_list[0] = re.sub("L", "", line_list[0])

        self.x1, self.y1 = current_pt
        self.x2 = float(line_list[0])
        self.y2 = float(line_list[1])
        self.end_coords: tuple = (self.x2, self.y2)

        self.Pt: bool = (self.x1 == self.x2) and (self.y1 == self.y2)
        self.v: bool = (self.x1 == self.x2) and (self.y1 != self.y2)
        self.h: bool = (self.x1 != self.x2) and (self.y1 == self.y2)
        self.I: bool = (self.x1 != self.x2) and (self.y1 != self.y2)

        # Determine plane to determine projection type


        # x1, y1, x2, and y2 form the bounding box
        # Find the lower bound for x and y
        # These evaluate differently if coordinates are relative vs absolute
        self.x_l: float = (self.x1<self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_l: float = (self.y1<self.y2)*self.y1 + (self.y1>=self.y2)*self.y2
        # Find the upper bound for x and y
        self.x_u: float = (self.x1>self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_u: float = (self.y1>self.y2)*self.y1 + (self.y1<=self.y2)*self.y2

        self.b_box: Tuple[Tuple[float]] = ((self.x1, self.y1), (self.x2, self.y2))

    def transform(self, x: float, y: float) -> tuple:
        return x - self.view_x_l, self.view_y_l - y

    def detransform(self, x: float, y: float) -> tuple:
        return x + self.view_x_l, self.view_y_l + y

    def convert_to_view_rel_coords(self) -> None:
        self.x1, self.y1 = self.transform(self.x1, self.y1)
        self.x2, self.y2 = self.transform(self.x2, self.y2)

        # Need to think about these ones - do x1, x2, y1, and y2 update immediately?
        self.x_l, self.y_l = self.transform(self.x_l, self.y_l)
        self.x_u, self.y_u = self.transform(self)

    def convert_to_abs_coords(self) -> None:
        self.x1, self.y1 = self.detransform(self.x1, self.y1)
        self.x2, self.y2 = self.detransform(self.x2, self.y2)

        # Need to think about these ones - do x1, x2, y1, and y2 update immediately?
        self.x_l, self.y_l = self.detransform(self.x_l, self.y_l)
        self.x_u, self.y_u = self.detransform(self)

In [ ]:
class SVG_Drawing:
    def __init__(self, file_path: str, angle_projection: str = 'First'):
        self.file_path: str = file_path
        self.angle_projection: str = angle_projection.lower()
        self.tree = ET.parse(file_path)
        self.root = self.tree.getroot()

        # Handling views
        # Absolute coordinates
        view_coords: set = set()
        self.line_elements: list = []
        # Looking to extract 'transform' attribute
        # The last two arguments in transform are the x translation and y translation, respectively
        # We skip the first elem because it seems like it's just a rectangle with no size
        for elem in self.root[3][0][1:]:
            matrix_list = re.split(",", elem.attrib['transform'])
            matrix_list[-1] = re.sub("\)", "", matrix_list[-1])
            view_coords.add((float(matrix_list[-2]), float(matrix_list[-1])))

            # Also keep track if the line element is a hidden line
            hidden = ("stroke-dasharray" in elem.attrib)

            # Extracting line segment information
            line_path = elem[0].attrib['d']
            if line_path is not None:
                # Split string by letters:
                #re.split(r'([A-Za-z])'
                pass

        # There should be 3 views in total
        assert len(view_coords) >= 3, "Not enough views"
        assert len(view_coords) == 3, "Too many views"




        # Classify and instantiate the views
        # Angle projection is assumed to be 1st Angle

        view_coords = sorted(list(view_coords))
        # Left view will be to the right of front view (so x will be greater)
        self.LeftView = View(view_coords[-1])
        # Bottom view will be on top of the front view (so y will be less, since y is inverted in SVG)
        self.BottomView = View(view_coords[0])
        # Front view is the only one left
        self.FrontView = View(view_coords[1])

        # Process view characteristics here
        for elem in self.root[3][0][1:]:

            path_text = elem[0].attrib['d']
            segments = re.findall(r'[A-Za-z][^A-Za-z]*', path_text)
            for seg in segments:
                current_pt: tuple = (0., 0.)
                if seg[0] == 'M':
                    current_pt = (Move(seg).end_coords)
                elif seg[0] == 'L':
                    self.line_elements.append(Line(seg, current_pt=current_pt, view_coords=self.FrontView.coords,\
                                                   hidden=hidden))
                    current_pt = (Line(seg, current_pt=current_pt, view_coords=self.FrontView.coords,\
                                                   hidden=hidden).end_coords)
                elif seg[0] == 'C':
                    self.line_elements.append(Curve(seg, current_pt=current_pt, view_coords=self.FrontView.coords, \
                                                   hidden=hidden))
                    current_pt = (Curve(seg, current_pt=current_pt, view_coords=self.FrontView.coords, \
                                                   hidden=hidden).end_coords)

In [ ]:
dwg = SVG_Drawing('00000145.svg')
dwg.line_elements